## Measureyes HTDetect.py Demo -- Acting on data extracted from AWS Rekognition

The `HTDetect` class extracts **Head Turns (HTs)** from data produced by the processing of a stored target video via AWS Rekognition. This class offers a set of parameters that allow researchers and consultants to "tune" qualifications defining an HT and related metrics, such as **Head-Turn Rate (HTR)**, for A/B tests.

To run this demo live, use the csv file found [here](https://github.com/sewald101/measureyes/blob/master/src/aws_rekognition/data/measureyes_0924_01.csv) and the queries found [here](https://github.com/sewald101/measureyes/blob/master/src/aws_rekognition/app/measureyes.sql) to set up a local Postgres/measureyes database and `Measureyes_0924_01` data table.  
  

#### First Step: Import module and initialize `HTDetect` class with target data (named here for unique video ID)

In [1]:
import numpy as np
import pandas as pd

from HTDetect import HTDetect

In [2]:
target_vid = 'Measureyes_0924_01'

demo_1 = HTDetect(target_vid) # Initialize HTDetect class with video ID.

#### Second Step: Execute `main()` method with optional parameters to qualify head turns; defaults shown below.
(Except default for `print_results=False`)

In [3]:
demo_1.main(yaw_threshold=45, # Degrees of head pose left and right of camera center within which to qualify as HT
            pitch_threshold=45, # Degrees, ditto: head pose up and down from camera center 
            dwell_threshold=1.5, # Seconds, minimum necessary of head turned toward camera to qualify as HT
            HT_break=1.5, # Seconds, minimum interruption of HT for single viewer to constitute a break bw HTs
            print_results=True # Print a summary of results
           )


 Total Persons Detected: _____166
 Total HTers: __________________5
 Total HTs Detected: __________21

 Head-Turn Rate: ___________0.030

 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
 Source Video ID: Measureyes_0924_01
 Head-Turn Parameters:
     Yaw Threshold (max allowable face angle left or right of camera): 45 degrees
     Pitch Threshold (max allowable angle up or down from camera): 45 degrees
     Minimum dwell to qualify: 1.5 seconds
     Minimum duration (break) between multiple HTs by single person: 1.5 seconds
 


#### Aggregate statistics are accessible through the following `HTDetect` attributes.

In [4]:
print("Count of distinct persons detected in video: ", demo_1.ttl_persons)
print("Count of distinct persons detected who made HTs: ", demo_1.HTers)
print("Head-Turn Rate, the ratio of HTers to total persons detected: {:.3f}".format(demo_1.HTR))
print("Count of HTs detected: {}".format(len(demo_1.HTs_df)))

Count of distinct persons detected in video:  166
Count of distinct persons detected who made HTs:  5
Head-Turn Rate, the ratio of HTers to total persons detected: 0.030
Count of HTs detected: 21


#### Data for HTs are returned in the `HTDetect.HTS_df` object.

HTs are ordered chronologically by HT_start, a timestamp of milliseconds from the start of the target video.  
Use Pandas `DataFrame.sort_values(by=col)` method to sort by dwell time or by person_index.

In [5]:
demo_1.HTs_df

,HT_start,HT_dwell,person_index
1,6381,5.005,33
2,9175,3.003,32
3,17976,3.420,34
4,27402,6.798,32
5,32198,2.586,46
6,37412,8.175,32
7,41207,3.003,34
8,47005,4.421,47
9,47213,4.213,32
10,60018,5.797,34


#### The `HTDetect` class provides other data products for the researcher's / consultant's reference:

1. `HTDetect.all_records_df`: a DataFrame of all original data (person-detection records) queried from the database.

In [6]:
print("Number of records: ", len(demo_1.all_records_df))
demo_1.all_records_df.head()

Number of records:  9069


,source_file,timestamp,person_index,face_yaw,face_pitch,face_box_top,face_box_left
0,Measureyes_0924_01_response_0001.json,0,0,NaN,NaN,NaN,NaN
1,Measureyes_0924_01_response_0001.json,41,0,NaN,NaN,NaN,NaN
2,Measureyes_0924_01_response_0001.json,125,0,NaN,NaN,NaN,NaN
3,Measureyes_0924_01_response_0001.json,166,0,NaN,NaN,NaN,NaN
4,Measureyes_0924_01_response_0001.json,250,0,NaN,NaN,NaN,NaN


2. `HTDetect.facing_camera_df` provides a DataFrame of all records in which a head pose (i.e., face) was detected.

In [7]:
print("Number of records: ", len(demo_1.facing_camera_df))
demo_1.facing_camera_df.head()

Number of records:  508


,timestamp,person_index,face_yaw,face_pitch
893,11386,19,-32.426006,-4.658849
2046,6172,31,-31.617493,-2.338707
2058,5797,32,-2.420268,0.076770
2059,6798,32,-14.524308,-2.389813
2061,8383,32,-60.660793,-12.552453


3. `HTDetect.filtered_by_pose` provides a DataFrame of all records that fell within specified pose (face_yaw, face_pitch) parameters

In [8]:
print("Number of records: ", len(demo_1.filtered_by_pose))
demo_1.filtered_by_pose.head()

Number of records:  444


,timestamp,person_index,face_yaw,face_pitch
893,11386,19,-32.426006,-4.658849
2046,6172,31,-31.617493,-2.338707
2058,5797,32,-2.420268,0.076770
2059,6798,32,-14.524308,-2.389813
2066,9175,32,-44.529732,-2.461496


**Tightening (or loosening) the paramaters to qualify HTs from the data is as simple as modifying the arguments in the `HTDetect.main()` method.**

Example 1: Let's say we only want to count a head turn where the viewers head "yaw" was no more than **30 degrees from camera center** (as opposed to the 45-degree default). Notice only 16 HTs detected under this tightened parameter as opposed to 21 HTs under the default setting.

In [9]:
demo_2 = HTDetect(target_vid)
demo_2.main(yaw_threshold=30, print_results=True)


 Total Persons Detected: _____166
 Total HTers: __________________5
 Total HTs Detected: __________16

 Head-Turn Rate: ___________0.030

 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
 Source Video ID: Measureyes_0924_01
 Head-Turn Parameters:
     Yaw Threshold (max allowable face angle left or right of camera): 30 degrees
     Pitch Threshold (max allowable angle up or down from camera): 45 degrees
     Minimum dwell to qualify: 1.5 seconds
     Minimum duration (break) between multiple HTs by single person: 1.5 seconds
 


Example 2: Or let's say we only want to count a head turn if the viewer was **looking toward the camera for a minimum of 3 seconds** (i.e., a 3-second "dwell" versus the 1.5 second default). Notice only 14 HTs registered under this tightened parameter.

In [10]:
demo_3 = HTDetect(target_vid)
demo_3.main(dwell_threshold=3, print_results=True)


 Total Persons Detected: _____166
 Total HTers: __________________4
 Total HTs Detected: __________14

 Head-Turn Rate: ___________0.024

 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
 Source Video ID: Measureyes_0924_01
 Head-Turn Parameters:
     Yaw Threshold (max allowable face angle left or right of camera): 45 degrees
     Pitch Threshold (max allowable angle up or down from camera): 45 degrees
     Minimum dwell to qualify: 3 seconds
     Minimum duration (break) between multiple HTs by single person: 1.5 seconds
 
